In [47]:
import os
import pandas as pd
import numpy as np
from ast import literal_eval
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras import Model, Sequential, Input
from tensorflow.keras.layers import InputLayer, Conv2D, Embedding, BatchNormalization, \
                              Dropout, Flatten, Dense, LSTM, SpatialDropout1D, Bidirectional, \
                                Conv1D, Softmax, MultiHeadAttention
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.optimizers import Nadam, Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


In [2]:
os.listdir('/network/scratch/j/jonathan.lim/IFT6759_Adv_Proj/quick-recipe/features/-CP9gKR9GR4/')

['master_features_with_label.csv',
 'master_features_with_ts_v2.csv',
 'master_features_v2.csv']

In [8]:
df = pd.read_csv('/network/scratch/j/jonathan.lim/IFT6759_Adv_Proj/quick-recipe/features/oJZUxU9szWA/master_features_with_label.csv', header=None)
# df = df.drop([0,1,2], axis='columns')
# df = df.dropna()
df = df[df[513] != -1.0]

tmp = pd.DataFrame()
for i in df[513].unique():
    # print(i)
    s_df = df[df[513] == i].reset_index()
    idx = random.randint(0, len(s_df) - 1)
    # print(idx, type(df.iloc[idx]))
    data = [tmp, pd.DataFrame([s_df.loc[idx].tolist()], columns=s_df.loc[idx].index)]
    tmp = pd.concat(data, ignore_index=True)

for i in tmp[513]:
    print(i)


0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
135.0


In [4]:
reviewed_df = pd.read_csv('/home/mila/s/subhrajyoti.dasgupta/quick-recipe/youcook2/reviewed_0812.csv')
# len(reviewed_df[reviewed_df['VideoUrl'].str[-11:] == 'Vy5ypC9_cOs'])

In [5]:
import random
import os

ROOT = '/network/scratch/j/jonathan.lim/IFT6759_Adv_Proj/quick-recipe/features/'

master_df = pd.DataFrame()
count = 0

for yt_id in os.listdir(ROOT):
    df = pd.read_csv(ROOT + yt_id + '/master_features_with_label.csv', header=None)
    # df = df.drop([0,1,2], axis='columns')
    # df = df.dropna()
    df = df[df[513] != -1.0]
    if len(df[513].unique()) != len(reviewed_df[reviewed_df['VideoUrl'].str[-11:] == yt_id]):
        print(yt_id)
        print(len(df[513].unique()), len(reviewed_df[reviewed_df['VideoUrl'].str[-11:] == yt_id]))
    for i in df[513].unique():
        # print(i)
        s_df = df[df[513] == i].reset_index()
        idx = random.randint(0, len(s_df) - 1)
        # print(idx, type(df.iloc[idx]))
        data = [master_df, pd.DataFrame([s_df.loc[idx].tolist()], columns=s_df.loc[idx].index)]
        master_df = pd.concat(data, ignore_index=True)
        count += 1
        # print(df.loc[idx].to_frame)

print(count)
master_df


okvTq8gtr8c
68 69
lBguj96fa5w
74 75
so-RuJQY1d0
85 86
-goI2-eJO1w
76 77
5nh2CP22dgY
101 102
oJZUxU9szWA
133 136
psXv6tX1GnA
77 79
_gG0As2y92g
72 73
btikV_DUoCM
83 86
15497


,index,0,1,2,3,4,5,6,7,8,...,505,506,507,508,509,510,511,512,513,514
0,11.0,0.425910,0.008941,0.643236,0.282954,0.199728,0.110655,0.139015,0.144657,0.229776,...,0.238769,0.111750,0.196669,0.139701,0.039804,0.148513,0.426560,6.278360,0.0,0.0
1,19.0,0.367234,0.031014,0.708423,0.357866,0.148172,0.142997,0.090542,0.135463,0.116519,...,0.216835,0.213623,0.047887,0.255960,0.125699,0.188846,0.386612,10.844440,1.0,0.0
2,33.0,0.656100,0.020784,0.369546,0.122344,0.289988,0.246644,0.260781,0.169352,0.255411,...,0.253150,0.237087,0.449494,0.120986,0.252464,0.075629,0.391871,18.835080,2.0,0.0
3,42.0,0.369797,0.013319,0.604215,0.168889,0.128159,0.117529,0.047085,0.178711,0.163233,...,0.300945,0.180592,0.057323,0.122506,0.007082,0.106830,0.420382,23.971920,3.0,0.0
4,44.0,0.353997,0.013369,0.572096,0.247803,0.134085,0.140081,0.050938,0.138815,0.182948,...,0.249933,0.076142,0.125779,0.131521,0.018362,0.181572,0.445160,25.113440,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15492,4423.0,0.293095,0.063478,0.097504,0.626769,0.051923,0.030981,0.724980,0.173229,0.072926,...,0.216833,0.007520,0.579740,0.339482,1.143520,0.116974,0.291777,232.276107,49.0,0.0
15493,4508.0,0.086693,0.115595,0.258870,0.435874,0.000375,0.128880,0.227669,0.304898,0.044436,...,0.389757,0.129661,0.042039,0.462857,0.266311,0.094459,0.329260,236.678073,50.0,0.0
15494,4589.0,0.182244,0.173184,0.268841,0.303563,0.043643,0.223073,0.244005,0.263544,0.220030,...,0.236928,0.010622,0.026865,0.226570,0.222489,0.332773,0.208293,240.920002,51.0,1.0
15495,4626.0,0.167179,0.150134,0.276072,0.275155,0.032595,0.219048,0.248460,0.232679,0.228654,...,0.268985,0.013566,0.024358,0.178112,0.203218,0.331389,0.188770,242.900935,52.0,0.0


In [6]:
master_df.groupby(514).count()

,index,0,1,2,3,4,5,6,7,8,...,504,505,506,507,508,509,510,511,512,513
514,,,,,,,,,,,,,,,,,,,,,
0.0,11928,11928,11928,11928,11928,11928,11928,11928,11928,11928,...,11928,11928,11928,11928,11928,11928,11928,11928,11928,11928
1.0,3569,3569,3569,3569,3569,3569,3569,3569,3569,3569,...,3569,3569,3569,3569,3569,3569,3569,3569,3569,3569


In [10]:
master_df.drop('index', axis='columns').to_csv('data.csv', header=False, index=False)

## Concating all features of a video together

In [29]:
import random
import os

ROOT = '/network/scratch/j/jonathan.lim/IFT6759_Adv_Proj/quick-recipe/features/'

master_df = pd.DataFrame()
count = 0

for yt_id in os.listdir(ROOT):
    df = pd.read_csv(ROOT + yt_id + '/master_features_with_label.csv', header=None)
    df = df[df[513] != -1.0]
    features = []
    labels = []
    for i in df[513].unique():
        # print(i)
        s_df = df[df[513] == i].reset_index()
        idx = random.randint(0, len(s_df) - 1)
        features.append(s_df.loc[idx].tolist()[1:-3])
        labels.append(s_df.loc[idx].tolist()[-1])
    data = [master_df, pd.DataFrame([[yt_id, features, labels]])]
    master_df = pd.concat(data, ignore_index=True)

print(count)
master_df


0


,0,1,2
0,Vy5ypC9_cOs,"[[0.32892459630966, 0.0015696668997407, 0.6470...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,bczCfVC6HYU,"[[0.077356286346912, 0.35245475172997, 0.18208...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
2,qaDMnLE6NBU,"[[0.39304235577583, 0.076984830200672, 0.17917...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,4nxbRG6-sfw,"[[0.12185056507587, 0.2575415968895, 0.0995274...","[0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ..."
4,C0gWH2E4lWg,"[[0.091066710650921, 0.49502757191658, 0.10811...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
347,zYMelEXKars,"[[0.30220186710358, 0.56805688142776, 0.044990...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ..."
348,SuK9T9KOBSs,"[[0.12748539447784, 0.14122919738293, 0.125872...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
349,lU28_L508vo,"[[0.36030226945877, 0.20773614943027, 0.130806...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
350,ueGKsfUW-PM,"[[0.79705458879471, 0.29097408056259, 0.099323...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [42]:
# len(master_df.iloc[0][1][7]) == len(master_df.iloc[0][2][7])

for i in tqdm(range(len(master_df))):
    if len(master_df.iloc[i][1]) != len(master_df.iloc[i][2]):
        print(master_df.iloc[i][0])


100%|██████████| 352/352 [00:00<00:00, 13235.87it/s]


In [43]:
master_df.to_csv('master_features.csv', header=False, index=False)

## Start of Modelling

In [2]:
splits = pd.read_csv('/network/scratch/j/jonathan.lim/IFT6759_Adv_Proj/quick-recipe/data/train_val_split.csv')
train_lst = splits[splits['Split']=='train']['VideoUrl'].str[-11:].to_list()
val_lst = splits[splits['Split']=='val']['VideoUrl'].str[-11:].to_list()

In [3]:
df = pd.read_csv('master_features.csv', header=None)
for i in tqdm(range(len(df))):
    df.loc[i][1] = literal_eval(df.loc[i][1])
    df.loc[i][2] = literal_eval(df.loc[i][2])

train_df = df[df[0].isin(train_lst)]
val_df = df[df[0].isin(val_lst)]

100%|██████████| 352/352 [00:27<00:00, 12.69it/s]


In [25]:
maxlen=133

train_x = pad_sequences(train_df[1], maxlen=maxlen, dtype='float')
print(train_x.shape)
val_x = pad_sequences(val_df[1], maxlen=maxlen, dtype='float')
print(val_x.shape)

train_y = pad_sequences(train_df[2], maxlen=maxlen, dtype='float', value=-1.0)
print(train_y.shape)
val_y = pad_sequences(val_df[2], maxlen=maxlen, dtype='float', value=-1.0)
print(val_y.shape)


(253, 133, 512)
(99, 133, 512)
(253, 133)
(99, 133)


In [66]:
model = Sequential()
model.add(InputLayer(input_shape=(maxlen,512)))
model.add(Conv1D(300, 3, padding='valid', activation='relu', strides=1))
model.add(Conv1D(150, 3, padding='valid', activation='relu', strides=1))
model.add(Conv1D(75, 3, padding='valid', activation='relu', strides=1))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(maxlen))

model.compile(optimizer='adam', loss=tf.keras.metrics.binary_crossentropy, metrics=[macro_f1])

model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_52 (Conv1D)          (None, 131, 300)          461100    
                                                                 
 conv1d_53 (Conv1D)          (None, 129, 150)          135150    
                                                                 
 conv1d_54 (Conv1D)          (None, 127, 75)           33825     
                                                                 
 flatten_18 (Flatten)        (None, 9525)              0         
                                                                 
 dense_36 (Dense)            (None, 64)                609664    
                                                                 
 dropout_18 (Dropout)        (None, 64)                0         
                                                                 
 dense_37 (Dense)            (None, 133)             

In [67]:
%%time
history = model.fit(train_x, train_y, epochs = 5, 
                    validation_data=(val_x, val_y)
                    )

Epoch 1/5
8/8 [==============================] - 2s 82ms/step - loss: -6.4166 - macro_f1: 0.0578 - val_loss: -7.1029 - val_macro_f1: 0.0178
Epoch 2/5
8/8 [==============================] - 0s 54ms/step - loss: -7.7033 - macro_f1: 0.0415 - val_loss: -8.1951 - val_macro_f1: 0.0180
Epoch 3/5
8/8 [==============================] - 0s 52ms/step - loss: -8.2804 - macro_f1: 0.0241 - val_loss: -8.5539 - val_macro_f1: 0.0052
Epoch 4/5
8/8 [==============================] - 0s 53ms/step - loss: -8.7798 - macro_f1: 0.0185 - val_loss: -8.7165 - val_macro_f1: 0.0159
Epoch 5/5
8/8 [==============================] - 0s 46ms/step - loss: -8.8635 - macro_f1: 0.0289 - val_loss: -8.8372 - val_macro_f1: 0.0137
CPU times: user 6.91 s, sys: 139 ms, total: 7.05 s
Wall time: 3.23 s


In [68]:
preds = model.predict(val_x)
preds = np.around(preds)
preds = preds.reshape(val_y.shape[0]*maxlen)
preds.sum()

4/4 [==============================] - 0s 11ms/step


-6261.0

In [69]:
preds.shape

(13167,)

In [12]:
model = Sequential()
model.add(InputLayer(input_shape=(maxlen,512)))
model.add(Conv1D(1, 3, padding='same', activation='relu', strides=1))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(maxlen, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 133, 1)            1537      
                                                                 
 flatten_2 (Flatten)         (None, 133)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8576      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 133)               8645      
                                                                 
Total params: 18,758
Trainable params: 18,758
Non-trainable params: 0
_________________________________________________________________


In [13]:
%%time
history = model.fit(train_x, train_y, epochs = 10, 
                    validation_data=(val_x, val_y)
                    )

Epoch 1/10
8/8 [==============================] - 1s 38ms/step - loss: 0.6903 - accuracy: 0.0079 - val_loss: 0.6845 - val_accuracy: 0.0202
Epoch 2/10
8/8 [==============================] - 0s 23ms/step - loss: 0.6528 - accuracy: 0.0119 - val_loss: 0.5727 - val_accuracy: 0.0202
Epoch 3/10
8/8 [==============================] - 0s 22ms/step - loss: 0.5281 - accuracy: 0.0040 - val_loss: 0.4120 - val_accuracy: 0.0101
Epoch 4/10
8/8 [==============================] - 0s 22ms/step - loss: 0.3919 - accuracy: 0.0079 - val_loss: 0.3131 - val_accuracy: 0.0101
Epoch 5/10
8/8 [==============================] - 0s 21ms/step - loss: 0.3101 - accuracy: 0.0079 - val_loss: 0.2650 - val_accuracy: 0.0000e+00
Epoch 6/10
8/8 [==============================] - 0s 24ms/step - loss: 0.2664 - accuracy: 0.0356 - val_loss: 0.2402 - val_accuracy: 0.0202
Epoch 7/10
8/8 [==============================] - 0s 23ms/step - loss: 0.2416 - accuracy: 0.0277 - val_loss: 0.2261 - val_accuracy: 0.0404
Epoch 8/10
8/8 [=======

In [16]:
model.save('cnn_merged_feat_v1')

2023-04-13 20:05:23.067407: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,64]
	 [[{{node inputs}}]]
2023-04-13 20:05:23.222033: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,64]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: cnn_merged_feat_v1/assets


INFO:tensorflow:Assets written to: cnn_merged_feat_v1/assets


In [14]:
preds = model.predict(val_x)
# preds = [round(x[0]) for x in preds]
preds = np.around(preds)
preds = preds.reshape(99*133)
preds.shape

4/4 [==============================] - 0s 4ms/step


(13167,)

In [15]:
from sklearn.metrics import classification_report
print(classification_report(val_y.reshape(99*133), preds))

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96     12106
         1.0       0.50      0.01      0.03      1061

    accuracy                           0.92     13167
   macro avg       0.71      0.51      0.49     13167
weighted avg       0.89      0.92      0.88     13167

